In [1]:
import time
import cv2
import pandas as pd
import tensorflow as tf
from tqdm import tqdm
from pixel2meshpy3tf12.models import GCN
from pixel2meshpy3tf12.fetcher import *

# Set random seed
seed = 1024
np.random.seed(seed)
tf.set_random_seed(seed)

# Settings
flags = tf.app.flags
FLAGS = flags.FLAGS
flags.DEFINE_string('f', '', 'kernel')
flags.DEFINE_string('data_list', 'pixel2mesh/hitoshi_utils/train_list.txt', 'Data list path.')
flags.DEFINE_float('learning_rate', 3e-5, 'Initial learning rate.')
flags.DEFINE_integer('epochs', 30, 'Number of epochs to train.')
flags.DEFINE_integer('hidden', 192, 'Number of units in  hidden layer.')
flags.DEFINE_integer('feat_dim', 963, 'Number of units in perceptual feature layer.')
flags.DEFINE_integer('coord_dim', 3, 'Number of units in output layer.') 
flags.DEFINE_float('weight_decay', 5e-6, 'Weight decay for L2 loss.')

# Define placeholders(dict) and model
num_blocks = 3
num_supports = 2
placeholders = {
    'features': tf.placeholder(tf.float32, shape=(None, 3)), # initial 3D coordinates
    'img_inp': tf.placeholder(tf.float32, shape=(224, 224, 3)), # input image to network
    'labels': tf.placeholder(tf.float32, shape=(None, 6)), # ground truth (point cloud with vertex normal)
    'support1': [tf.sparse_placeholder(tf.float32) for _ in range(num_supports)], # graph structure in the first block
    'support2': [tf.sparse_placeholder(tf.float32) for _ in range(num_supports)], # graph structure in the second block
    'support3': [tf.sparse_placeholder(tf.float32) for _ in range(num_supports)], # graph structure in the third block
    'faces': [tf.placeholder(tf.int32, shape=(None, 4)) for _ in range(num_blocks)], # helper for face loss (not used)
    'edges': [tf.placeholder(tf.int32, shape=(None, 2)) for _ in range(num_blocks)], # helper for normal loss
    'lape_idx': [tf.placeholder(tf.int32, shape=(None, 10)) for _ in range(num_blocks)], # helper for laplacian regularization
    'pool_idx': [tf.placeholder(tf.int32, shape=(None, 2)) for _ in range(num_blocks-1)] # helper for graph unpooling
}

model = GCN(placeholders, logging=True)

Instructions for updating:
Use tf.initializers.variance_scaling instead with distribution=uniform to get equivalent behavior.


/usr/local/lib/python3.5/dist-packages/tensorflow/python/ops/gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [2]:
def construct_feed_dict(pkl, placeholders):
    coord, pool_idx, faces, lape_idx,  = pkl[0], pkl[4], pkl[5], pkl[7]
    edges = []
    for i in range(1,4):
        adj = pkl[i][1]
        edges.append(adj[0])
    feed_dict = dict()
    feed_dict.update({placeholders['features']: coord})
    feed_dict.update({placeholders['edges'][i]: edges[i] for i in range(len(edges))})
    feed_dict.update({placeholders['faces'][i]: faces[i] for i in range(len(faces))})
    feed_dict.update({placeholders['pool_idx'][i]: pool_idx[i] for i in range(len(pool_idx))})
    feed_dict.update({placeholders['lape_idx'][i]: lape_idx[i] for i in range(len(lape_idx))})
    feed_dict.update({placeholders['support1'][i]: pkl[1][i] for i in range(len(pkl[1]))})
    feed_dict.update({placeholders['support2'][i]: pkl[2][i] for i in range(len(pkl[2]))})
    feed_dict.update({placeholders['support3'][i]: pkl[3][i] for i in range(len(pkl[3]))})
    return feed_dict

# Load data
data = DataFetcher(FLAGS.data_list)
data.setDaemon(True)
data.start()
train_number = data.number
# Initialize session
config=tf.ConfigProto()
config.gpu_options.allow_growth=True
config.allow_soft_placement=True
sess = tf.Session(config=config)
sess.run(tf.global_variables_initializer())
model.load(sess)

INFO:tensorflow:Restoring parameters from pixel2meshpy3tf12/utils/checkpoint/gcn.ckpt
Model restored from file: pixel2meshpy3tf12/utils/checkpoint/gcn.ckpt


In [3]:
# Construct feed dictionary
pkl = pickle.load(open('pixel2meshpy3tf12/utils/ellipsoid/info_ellipsoid.dat', 'rb'),encoding='bytes')
feed_dict = construct_feed_dict(pkl, placeholders)
# Train model
train_loss = open('pixel2mesh/utils/record_training_loss.log', 'a')
train_loss.write('Start training, lr =  %f\n'%(FLAGS.learning_rate))

31

In [ ]:
start = time.time()
for epoch in tqdm(range(FLAGS.epochs)):
    all_loss = np.zeros(train_number, dtype='float32') 
    for iters in range(train_number):
        img_inp, y_train, data_id = data.fetch()
        feed_dict.update({placeholders['img_inp']: img_inp})
        feed_dict.update({placeholders['labels']: y_train})
        _, dists,out1,out2,out3 = sess.run([model.opt_op,model.loss,model.output1,model.output2,model.output3], feed_dict=feed_dict)
        all_loss[iters] = dists
        mean_loss = np.mean(all_loss[np.where(all_loss)])
        if (iters+1) % 32 == 0:
            print('Epoch %d, Iteration %d'%(epoch + 1,iters + 1))
            print('Mean loss = %f, iter loss = %f, %d'%(mean_loss,dists,data.queue.qsize()))
    model.save(sess)
    train_loss.write('Epoch %d, loss %f\n'%(epoch+1, mean_loss))
    train_loss.flush()

end = time.time()
print(end - start)
pd.DataFrame([{'runtime':end-start}]).to_csv('runtime.csv')
#data.shutdown()
#print 'Training Finished!'

  0%|          | 0/30 [00:00<?, ?it/s]

Epoch 1, Iteration 32
Mean loss = 3.886297, iter loss = 4.925266, 64
Epoch 1, Iteration 64
Mean loss = 4.231776, iter loss = 3.963134, 64
Epoch 1, Iteration 96
Mean loss = 4.134262, iter loss = 3.134963, 64
Epoch 1, Iteration 128
Mean loss = 4.178844, iter loss = 4.372570, 64
Epoch 1, Iteration 160
Mean loss = 4.176582, iter loss = 4.568299, 64
Epoch 1, Iteration 192
Mean loss = 4.172799, iter loss = 2.701644, 64
Epoch 1, Iteration 224
Mean loss = 4.171677, iter loss = 4.608515, 64
Epoch 1, Iteration 256
Mean loss = 4.189815, iter loss = 3.872772, 64
Epoch 1, Iteration 288
Mean loss = 4.227479, iter loss = 5.160542, 64
Epoch 1, Iteration 320
Mean loss = 4.213541, iter loss = 2.238525, 64
Epoch 1, Iteration 352
Mean loss = 4.205786, iter loss = 4.477055, 64
Epoch 1, Iteration 384
Mean loss = 4.183309, iter loss = 3.808223, 64
Epoch 1, Iteration 416
Mean loss = 4.192954, iter loss = 5.067133, 64
Epoch 1, Iteration 448
Mean loss = 4.218174, iter loss = 4.147779, 64
Epoch 1, Iteration 480


Epoch 1, Iteration 3744
Mean loss = 4.283196, iter loss = 5.532105, 64
Epoch 1, Iteration 3776
Mean loss = 4.281957, iter loss = 2.543175, 64
Epoch 1, Iteration 3808
Mean loss = 4.282546, iter loss = 3.550156, 64
Epoch 1, Iteration 3840
Mean loss = 4.283369, iter loss = 4.792110, 64
Epoch 1, Iteration 3872
Mean loss = 4.288104, iter loss = 3.996112, 64
Epoch 1, Iteration 3904
Mean loss = 4.284762, iter loss = 4.228325, 64
Epoch 1, Iteration 3936
Mean loss = 4.285182, iter loss = 5.439226, 64
Epoch 1, Iteration 3968
Mean loss = 4.287611, iter loss = 6.259146, 64
Epoch 1, Iteration 4000
Mean loss = 4.292322, iter loss = 4.340303, 64
Epoch 1, Iteration 4032
Mean loss = 4.290023, iter loss = 5.389574, 64
Epoch 1, Iteration 4064
Mean loss = 4.291143, iter loss = 3.874549, 64
Epoch 1, Iteration 4096
Mean loss = 4.292312, iter loss = 4.594769, 64
Epoch 1, Iteration 4128
Mean loss = 4.291422, iter loss = 3.023814, 64
Epoch 1, Iteration 4160
Mean loss = 4.289752, iter loss = 7.191788, 64
Epoch 

Epoch 1, Iteration 7456
Mean loss = 4.338800, iter loss = 5.615801, 64
Epoch 1, Iteration 7488
Mean loss = 4.338847, iter loss = 4.198309, 64
Epoch 1, Iteration 7520
Mean loss = 4.338146, iter loss = 3.648307, 64
Epoch 1, Iteration 7552
Mean loss = 4.340319, iter loss = 5.235971, 64
Epoch 1, Iteration 7584
Mean loss = 4.341838, iter loss = 4.085246, 64
Epoch 1, Iteration 7616
Mean loss = 4.341966, iter loss = 7.312643, 64
Epoch 1, Iteration 7648
Mean loss = 4.342413, iter loss = 3.864293, 64
Epoch 1, Iteration 7680
Mean loss = 4.341717, iter loss = 7.200637, 64
Epoch 1, Iteration 7712
Mean loss = 4.342473, iter loss = 4.665172, 64
Epoch 1, Iteration 7744
Mean loss = 4.341554, iter loss = 4.432621, 64
Epoch 1, Iteration 7776
Mean loss = 4.343431, iter loss = 7.027673, 64
Epoch 1, Iteration 7808
Mean loss = 4.343286, iter loss = 5.480428, 64
Epoch 1, Iteration 7840
Mean loss = 4.343264, iter loss = 4.677518, 64
Epoch 1, Iteration 7872
Mean loss = 4.341832, iter loss = 3.900995, 64
Epoch 